In [ ]:
#!pip install azure-storage-blob azure-identity
from azure.storage.blob import BlobServiceClient
import requests
import json
import pandas as pd


class MakeApiCall:
  
    def __init__(self,api,payload,):
        self.api=api
        self.payload=payload



    def make_api_call(self,header,param,method):
        header["Content-Type"]="application/json"
        header["Accept"]="application/json"
        if method=="GET":
            response = requests.get(f"{self.api}",params=param, headers=header)
            print(response.url)

        else:
            response = requests.post(f"{self.api}", data=self.payload, headers=header)
        
        if response.status_code == 200:
            print("sucessfully fetched the data with parameters provided")
            return response.json()
        else:
            print(f"Hello person, there's a {response.status_code} error with your request")
            
        
      

if __name__ == "__main__":
    username = input("Enter your Username")
    password =input("Enter Your password")
    obj_id=""
    header={}
    payload = {
            "username": username,
            "password": password
        }
    payload_json = json.dumps(payload)
    api_call = MakeApiCall("https://dm-us.informaticacloud.com/saas/public/core/v3/login",payload_json)
    login_response=api_call.make_api_call(header,{},"POST")
    user_info=login_response["userInfo"]
    session_id=user_info["sessionId"]
    header["INFA-SESSION-ID"]=session_id 
    api_call_2 = MakeApiCall("https://usw5.dm-us.informaticacloud.com/saas/public/core/v3/objects",{})
    object_response=api_call_2.make_api_call(header,{"q":"type=='PROJECT'"},"GET")
    object_list=object_response["objects"]
    obj_list=[]
    for obj in object_list:
            obj_list.append(obj["id"])

    dict_org={'Org_id':obj_list}
    df = pd.DataFrame(dict_org)
    df.to_csv('Org_info.csv')

# Azure Blob containe creation--

storag_connection_string = "DefaultEndpointsProtocol=https;AccountName=farazstorage;AccountKey=afu9JnfeN6u0a1I9NtZHcx9+J3WGsRZll4f9hpoZX9cJXBpPmlsQDnk+EpWOybONg52JzViXu20g+AStS+W5dQ==;EndpointSuffix=core.windows.net"
blob_service_client=BlobServiceClient.from_connection_string(storag_connection_string)

#create container
container_name="faraz"
blob_service_client.create_container(container_name)



# Uploading file to blob

file_path="/content/Org_info.csv"
blob_obj=blob_service_client.get_blob_client(container=container_name,blob=file_path)
blob_obj.upload_blob(file_path)
print("File succesfully uploaded")

sucessfully fetched the data with parameters provided
https://usw5.dm-us.informaticacloud.com/saas/public/core/v3/objects?q=type%3D%3D%27PROJECT%27
sucessfully fetched the data with parameters provided
File succesfully uploaded
